<a href="https://colab.research.google.com/github/singhsneha99/BDS_Predictive/blob/main/Combined.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys,os
# Block which runs on both Google Colab and Local PC without any modification
if 'google.colab' in sys.modules:    
    project_path = "/content/drive/My Drive/"
    # Google Colab lib
    from google.colab import drive
    # Mount the drive
    drive.mount('/content/drive/', force_remount=True)
    sys.path.append(project_path)
    %cd $project_path

# Let's look at the sys path
print('Current working directory', os.getcwd())

Mounted at /content/drive/
/content/drive/My Drive
Current working directory /content/drive/My Drive


LINEAR REGRESSION:

In [3]:
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Read the data
rest = pd.read_csv('/content/drive/My Drive/TimeSeries_Data/Restaurants_time_series.csv')
cb = pd.read_csv('/content/drive/My Drive/TimeSeries_Data/Citibike_time_series.csv')
com = pd.read_csv('/content/drive/My Drive/TimeSeries_Data/Complaints_time_series.csv')
ev = pd.read_csv('/content/drive/My Drive/TimeSeries_Data/Evictions_time_series.csv')
ob = pd.read_csv('/content/drive/My Drive/TimeSeries_Data/Operating_Businesses_time_series.csv')
hi=pd.read_csv('/content/drive/My Drive/TimeSeries_Data/health_inspections_time_series.csv')

In [4]:
# Convert yr-month to a datetime object
rest['yr-month'] = pd.to_datetime(rest['yr-month'], format='%Y-%m')

cb['yr-month'] = pd.to_datetime(cb['yr-month'], format='%Y-%m')
com['yr-month'] = pd.to_datetime(com['yr-month'], format='%Y-%m')
ev['yr-month'] = pd.to_datetime(ev['yr-month'], format='%Y-%m')
ob['yr-month'] = pd.to_datetime(ob['yr-month'], format='%Y-%m')
hi['yr-month'] = pd.to_datetime(hi['yr-month'], format='%Y-%m')





In [5]:
# Rename columns to 'tract_rest', 'tract_cb', 'tract_com', 'tract_ev', 'tract_ob', 'tract_hi'
rest = rest.rename(columns={'tract': 'tract_rest'})
cb = cb.rename(columns={'tract': 'tract_cb'})
com = com.rename(columns={'tract': 'tract_com'})
ev = ev.rename(columns={'tract': 'tract_ev'})
ob = ob.rename(columns={'tract': 'tract_ob'})
hi = hi.rename(columns={'target': 'tract_hi'})
hi = hi.rename(columns={'avg_sales': 'SALE_PRICE'})


In [6]:
com.shape

(888, 5)

In [7]:
cb.dtypes

yr-month            datetime64[ns]
tract_1000_grp             float64
SALE_PRICE                 float64
counts_per_block             int64
tract_cb                   float64
dtype: object

In [8]:
com.dtypes

yr-month            datetime64[ns]
tract_1000_grp             float64
SALE_PRICE                 float64
tract_com                  float64
counts_per_block           float64
dtype: object

In [9]:
com.shape, cb.shape

((888, 5), (753, 5))

In [11]:

com

,yr-month,tract_1000_grp,SALE_PRICE,tract_com,counts_per_block
0,2014-01-01,1.0,1.217243e+06,1469.0,1469.0
1,2014-01-01,2.0,1.121723e+06,2843.0,2843.0
2,2014-01-01,3.0,6.194213e+05,1158.0,1158.0
3,2014-01-01,4.0,6.203758e+05,666.0,666.0
4,2014-01-01,5.0,6.229657e+05,628.0,628.0
...,...,...,...,...,...
883,2023-03-01,4.0,8.602889e+05,1880.0,1880.0
884,2023-03-01,5.0,8.808296e+05,1622.0,1622.0
885,2023-03-01,6.0,9.306053e+05,1640.0,1640.0
886,2023-03-01,7.0,6.492544e+05,514.0,514.0


In [12]:
hi.rename(columns={'tract_group':'tract_1000_grp'},inplace=True)


In [21]:
# Combine all datasets on column 'yr_month', only keeping values which are present for all columns
df = pd.merge(cb, com[['yr-month','tract_1000_grp', 'tract_com']], on=['yr-month','tract_1000_grp'], how='left')
df = pd.merge(df, ev[['yr-month','tract_1000_grp', 'tract_ev']], on=['yr-month','tract_1000_grp'], how='left')
df = pd.merge(df, ob[['yr-month','tract_1000_grp', 'tract_ob']], on=['yr-month','tract_1000_grp'], how='left')
df = pd.merge(df, hi[['yr-month','tract_1000_grp', 'tract_hi']], on=['yr-month','tract_1000_grp'], how='left')

df.shape


(753, 9)

In [22]:
df.isna().sum()

yr-month              0
tract_1000_grp        0
SALE_PRICE            0
counts_per_block      0
tract_cb              0
tract_com           109
tract_ev            501
tract_ob              0
tract_hi            312
dtype: int64

In [24]:
df.head()

,yr-month,tract_1000_grp,SALE_PRICE,counts_per_block,tract_cb,tract_com,tract_ev,tract_ob,tract_hi
0,2014-01-01,1.0,1.217243e+06,825,231762.0,1469.0,NaN,118.0,NaN
1,2014-10-01,1.0,1.177936e+06,895,639295.0,1612.0,NaN,96.0,NaN
2,2014-11-01,1.0,1.103000e+06,698,410588.0,1398.0,NaN,67.0,NaN
3,2014-12-01,1.0,1.106756e+06,840,307169.0,1357.0,NaN,93.0,NaN
4,2014-02-01,1.0,1.120497e+06,657,175893.0,1310.0,NaN,123.0,NaN


In [23]:
df.head()
df.to_csv('/content/drive/My Drive/TimeSeries_Data/merged_test.csv',index=False)


In [41]:
df3=df

In [26]:
df.isna().sum()

yr-month              0
tract_1000_grp        0
SALE_PRICE            0
counts_per_block      0
tract_cb              0
tract_com           109
tract_ev            501
tract_ob              0
tract_hi            312
dtype: int64

In [36]:
df2 = pd.merge(cb, com[['yr-month','tract_1000_grp', 'tract_com']], on=['yr-month','tract_1000_grp'], how='left')
df2 = pd.merge(df2, ob[['yr-month','tract_1000_grp', 'tract_ob']], on=['yr-month','tract_1000_grp'], how='left')
#df2 = pd.merge(df2, rest[['yr-month','tract_1000_grp', 'tract_rest']], on=['yr-month','tract_1000_grp'], how='left')

df2.shape

(753, 7)

In [37]:
df2.isna().sum()

yr-month              0
tract_1000_grp        0
SALE_PRICE            0
counts_per_block      0
tract_cb              0
tract_com           109
tract_ob              0
dtype: int64

In [38]:
df2.dropna(axis=0, how='any', inplace=True)


In [39]:
df2

,yr-month,tract_1000_grp,SALE_PRICE,counts_per_block,tract_cb,tract_com,tract_ob
0,2014-01-01,1.0,1.217243e+06,825,231762.0,1469.0,118.0
1,2014-10-01,1.0,1.177936e+06,895,639295.0,1612.0,96.0
2,2014-11-01,1.0,1.103000e+06,698,410588.0,1398.0,67.0
3,2014-12-01,1.0,1.106756e+06,840,307169.0,1357.0,93.0
4,2014-02-01,1.0,1.120497e+06,657,175893.0,1310.0,123.0
...,...,...,...,...,...,...,...
639,2023-02-01,6.0,8.991405e+05,148,95658.0,1299.0,12.0
640,2023-03-01,6.0,9.306053e+05,228,118620.0,1640.0,18.0
641,2023-01-01,7.0,7.538367e+05,110,206.0,528.0,2.0
642,2023-02-01,7.0,6.872055e+05,99,586.0,377.0,2.0


In [40]:
df2.to_csv('/content/drive/My Drive/TimeSeries_Data/larger_merged.csv',index=False)

In [42]:
df3.dropna(axis=0, how='any', inplace=True)

In [43]:
df3.shape

(209, 9)

In [44]:
df3.to_csv('/content/drive/My Drive/TimeSeries_Data/nonull.csv',index=False)